# Building a NanoGPT from Scratch: Attempting to Understand the Transformer Architecture
## 0. Introduction
My journey began in late 2024/early 2025 when I discovered Andrej Karpathy's video **Let's build GPT: from scratch, in code, spelled out**. The two-hour deep dive provided a clear and, low-level overview of the architecture. While I grasped the general framework of the Transformer, the inner workings of the Attention mechanism remained opaque.

The challenge wasn't just the architecture itself; it was my **foundation**. My memories of undergraduate linear algebra, & machine learning were nonexistent, as if by now those tokens were outside of my context window. necessitating a serious refresh on core concepts like **matrix multiplication, dot product, loss function, backpropagation, and tensor dimensionality**.

Throughout the year, I repeatedly returned to Karpathy’s explanation. **Each time, I peeled back a new layer of complexity**, steadily improving my grasp of the Transformer's operation. What started as a vague concept slowly transformed (hehe) into a detailed understanding.

This document is the culmination of that effort my attempt to synthesize everything I’ve learned about how these **"thinking machines"** work. I aimed for clarity and depth, and I hope it proves genuinely useful to others starting this same path.

This notebook provides a technical overview and guide for a minimal implementation of a Generative Pre-trained Transformer (**NanoGPT**), built using PyTorch. This project serves as a clear, educational reference for understanding the core components of modern large language models, from data preparation to the self-attention mechanism and text generation.

---

## 1. Data Acquisition and Character-Level Tokenization

The first step in any language modeling project is preparing the text data. For this notebook, we used a public domain work of literature: **Don Quijote de la Mancha**.

### Data Loading and Tokenization
The text was downloaded directly from Project Gutenberg using `curl`. A simple **character-level tokenizer** was implemented by mapping every unique character to an integer ID.

* The **vocabulary size** ($\boldsymbol{V}$ or `vocab_size`) is determined by the number of unique characters.
* The entire corpus is encoded into a long list of integers.

### Data Preparation and Dimensions
The encoded corpus is split into training and validation sets (90/10). For training, we sample batches of input sequences ($X$) and target sequences ($Y$).

Let $B$ be the batch size (which is implicitly set to `n_embd = 64` in the `sample_data` call for the loss estimate) and $T$ be the context length (`block_size = 32`).

* **Input Data:** The input batch $X$ is a tensor of shape $(B, T)$.
* **Target Data:** The target batch $Y$ is also a tensor of shape $(B, T)$, where $Y_{i, t}$ is the correct prediction for the token $X_{i, t+1}$.

### Sources

* **Attention Is All You Need**
* **LLM Visualization**
* **Understanding and Coding the Self-Attention Mechanism of Large Language Models From Scratch**
* **Andrej Karpathy's makemore/nanoGPT Video**
* **Language Models are Unsupervised Multitask Learners (GPT-2 Paper)**
* **Wikipedia: Byte-pair encoding**

In [1]:
!curl -O https://www.gutenberg.org/files/2000/2000-0.txt

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 2173k  100 2173k    0     0  1801k      0  0:00:01  0:00:01 --:--:-- 1802k


In [1]:
%pip install numpy
%pip install torch

/Users/stephan.meza/ToyGPT/toy_gpt/bin/python3: No module named pip
Note: you may need to restart the kernel to use updated packages.
/Users/stephan.meza/ToyGPT/toy_gpt/bin/python3: No module named pip
Note: you may need to restart the kernel to use updated packages.


In [3]:
import torch
import numpy

In [4]:
with open("./2000-0.txt", "r") as f:
    contents = f.read()

In [5]:
num_tokens = 500
offset = 3
print(contents[num_tokens*offset:num_tokens*offset+num_tokens])

noso y grande escrutinio que el cura y el
barbero hicieron en la librería de nuestro ingenioso hidalgo

De la segunda salida de nuestro buen caballero don
Quijote de la Mancha

Del buen suceso que el valeroso don Quijote tuvo en
la espantable y jamás imaginada aventura de los molinos de viento, con
otros sucesos dignos de felice recordación

Donde se concluye y da fin a la estupenda batalla que
el gallardo vizcaíno y el valiente manchego tuvieron

De lo que más le avino a don Quijote con el vizc


### Character-Level Tokenization and Vocab Creation

We first create a simple **character-level tokenizer**. This approach is simple and highly robust (never fails on an OOV word) but is computationally inefficient for large models due to sequence bloat.

1.  **Extract Vocabulary:** Find all unique characters in the corpus.
2.  **Mapping:** Create lookup tables (`stoi` for string-to-integer and `itos` for integer-to-string).

In [6]:
# create a mapping from characters to integers
chars = sorted(list(set(contents)))
vocab_size = len(chars)
stoi = { ch:i for i,ch in enumerate(chars) }

itos = { i:ch for i,ch in enumerate(chars) }
encode = lambda s: [stoi[c] for c in s] # encoder: take a string, output a list of integers
decode = lambda l: ''.join([itos[i] for i in l]) # decoder: take a list of integers, output a string

In [7]:
a = "hello!"
print(encode(a))
print(decode(encode(a)))

[64, 61, 68, 68, 71, 2]
hello!


### Batching Function

The **Transformer** operates on fixed-length chunks. We define a function to sample random batches of input ($X$) and target ($Y$) sequences.

* **Context Length ($\boldsymbol{T}$):** `block_size` (e.g., 32) is the maximum sequence length the model processes at once.
* **Batch Size ($\boldsymbol{B}$):** The number of independent sequences processed in parallel.

The target $\boldsymbol{Y}$ is simply the input $\boldsymbol{X}$ shifted one position to the right, as the model predicts the next token in the sequence.

In [8]:
def split_data(training_data: str, split_ratio: float = 0.9) -> tuple[str,str]:
    training_data_length = len(training_data)
    marker = int(split_ratio * training_data_length)
    return training_data[:marker], training_data[marker:]

In [9]:
import torch
def sample_data(data, len_samples:int=16, num_samples:int=8) -> torch.Tensor:
    len_data = len(data)
    random_indices = torch.randint(0, len_data - len_samples, (num_samples,))
    sample_x = torch.stack([
        torch.tensor(data[r_ind:r_ind+len_samples]) 
        for r_ind in random_indices
    ])
    sample_y = torch.stack([
        torch.tensor(data[r_ind+1:r_ind+1+len_samples]) 
        for r_ind in random_indices
    ])
    return sample_x, sample_y

In [10]:
encoded_contents = encode(contents)
training_data, validation_data = split_data(encoded_contents)

## 2. The Transformer Block and Attention Mechanism

The core of the NanoGPT is the **Transformer Block**, based on the seminal "Attention Is All You Need" paper. This is a decoder-only architecture, characteristic of GPT models.

### The NanoGPT Architecture
The model uses several learned components to transform the input:
1.  **Token Embeddings:** Maps the input IDs $(B, T)$ to a continuous space of size $C$ (`embedding_dim = 2048`).
    $$\text{Token Embeddings}: (B, T) \rightarrow (B, T, C)$$
2.  **Positional Embeddings:** A learned encoding added to the token embeddings to provide sequential context, as attention itself is order-agnostic.
3.  **Final Input Vector:** The sum of the token and positional embeddings forms the final input vector to the blocks.

### Causal Self-Attention and Matrix Sizes
The `AttentionHead` implements the scaled dot-product attention, modified for causality (GPT-style).

Let $H$ be the size of a single attention head (`head_size`). $H = C / N_h$, where $N_h$ is the number of heads (`num_heads = 12`).

1.  **Linear Projections (Q, K, V):** The input vector $x$ (shape $(B, T, C)$) is projected into Query ($\boldsymbol{Q}$), Key ($\boldsymbol{K}$), and Value ($\boldsymbol{V}$) matrices.
    $$\boldsymbol{Q}, \boldsymbol{K}, \boldsymbol{V} = W_Q(x), W_K(x), W_V(x) \rightarrow (B, T, H)$$
2.  **Attention Logits ($\Omega$):** The raw attention scores ($\Omega$) are calculated by the dot product of $Q$ and $K^T$. This score determines how much token $i$ should pay attention to token $j$.
    $$\Omega = \boldsymbol{Q}\boldsymbol{K}^T \rightarrow (B, T, H) \times (B, H, T) \rightarrow (B, T, T)$$
3.  **Scaling and Masking:** $\Omega$ is scaled by $\frac{1}{\sqrt{H}}$ to stabilize gradients. A lower-triangular mask (`tril`) is applied, setting entries corresponding to future tokens to $-\infty$. This is the **causal mask**, enforcing that the prediction for token $t$ can only depend on tokens $1$ through $t$.
4.  **Attention Weights ($W$):** The masked logits are passed through a softmax function, resulting in the attention weights $W$. These weights are the probability distribution over past tokens for the current token.
    $$W = \text{softmax}(\Omega_{\text{masked}} / \sqrt{H}) \rightarrow (B, T, T)$$
5.  **Context Vector (Output):** The weights $W$ are multiplied by the Value matrix $V$ to create the context vector, which is the output of the attention head.
    $$\text{Output} = W\boldsymbol{V} \rightarrow (B, T, T) \times (B, T, H) \rightarrow (B, T, H)$$

The **Multi-Head Attention** module runs $N_h$ heads in parallel, concatenates their outputs (shape $(B, T, N_h \cdot H)$), and projects the result back to $C$ using a final linear layer.

The **TransformerBlock** wraps this, applying Layer Normalization (Post-LN style) and **residual connections**: $x' = x + \text{LayerNorm}(\text{MultiHead}(x))$. This structure ensures gradients can flow easily through the network.

---

In [11]:
import torch.nn as nn
class AttentionHead(nn.Module):
    def __init__(self, n_embd:int, head_size:int, block_size:int, dropout:int=0):
        super().__init__()
        self.head_size = head_size
        self.W_query = nn.Linear(n_embd, head_size, bias=False)
        self.W_key = nn.Linear(n_embd, head_size, bias=False)
        self.W_value = nn.Linear(n_embd, head_size, bias=False)
        self.register_buffer('tril', torch.tril(torch.ones(block_size, block_size)))
        self.dropout = nn.Dropout(dropout)
        
    def forward(self, x: torch.Tensor) -> torch.Tensor:
        B,T,C = x.shape
        keys = self.W_key(x)
        queries = self.W_query(x)
        values = self.W_value(x)
        omega = queries @ keys.mT # * C**-0.5

        omega_masked = omega.masked_fill(self.tril[:T, :T] == 0, float('-inf'))

        attention_weights = nn.functional.softmax(omega_masked / self.head_size**0.5, dim=-1)
        wei = self.dropout(attention_weights)

        context_vector = wei @ values # (num_samples, num_words, num_embeddings_per_word)

        return context_vector

class MultiAttentionHead(nn.Module):
    def __init__(self, num_heads:int, n_embd:int, head_size:int, block_size:int, dropout:int=0.0):
        super().__init__()
        self.attention_blocks = nn.ModuleList([AttentionHead(n_embd, head_size, block_size) for _ in range(num_heads)])
        self.linear_projection = nn.Linear(head_size * num_heads, n_embd)
        self.dropout = nn.Dropout(dropout)
        
    def forward(self, x: torch.Tensor) -> torch.Tensor:
        attention_weights = torch.cat([head(x) for head in self.attention_blocks], dim=-1)
        lin = self.linear_projection(attention_weights)
        
        return self.dropout(lin)

class FeedForwardNetwork(nn.Module):
    def __init__(self, n_embd:int, block_size:int, dropout:int=0.0):
        super().__init__()
        self.ln1 = nn.Linear(n_embd, 4 * n_embd)
        self.relu = nn.ReLU()
        self.ln2 = nn.Linear(4 * n_embd, n_embd)
        self.dropout = nn.Dropout(dropout)

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        x = self.ln1(x)
        x = self.relu(x)
        x = self.ln2(x)
        return self.dropout(x)

class TransformerBlock(nn.Module):
    def __init__(self, num_heads:int, n_embd:int, head_size:int, block_size:int):
        super().__init__()
        self.multi_head_attention = MultiAttentionHead(num_heads, n_embd, head_size, block_size)
        self.ffn = FeedForwardNetwork(n_embd, block_size)
        self.ln1 = nn.LayerNorm(n_embd)
        self.ln2 = nn.LayerNorm(n_embd)

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        #x = x + self.multi_head_attention(self.ln1(x))
        #x = x + self.ffn(self.ln2(x))

        x = x + self.ln1(self.multi_head_attention(x))
        x = x + self.ln2(self.ffn(x))
        return x


In [12]:
# A very simple and stupid GPT model!
class NanoGPT(nn.Module):
    def __init__(self, num_layers:int, vocab_size:int, embedding_dim:int, num_heads:int, block_size:int):
        super().__init__()
        head_size = embedding_dim // num_heads

        self.block_size = block_size
        self.embedding = nn.Embedding(vocab_size,embedding_dim)
        self.positional_embeddings = nn.Embedding(block_size,embedding_dim)

        self.blocks = nn.Sequential(*[TransformerBlock(num_heads, embedding_dim, head_size, block_size) for _ in range(num_layers)])
    
        self.ln1 = nn.LayerNorm(embedding_dim)
        self.lm_head = nn.Linear(embedding_dim, vocab_size)
        
    def forward(self, input_ids: torch.Tensor, targets: torch.Tensor|None=None) -> torch.Tensor:
        B,T = input_ids.shape
        word_embeddings = self.embedding(input_ids)
        pos = torch.arange(T)

        positional_embeddings = self.positional_embeddings(pos)

        final_vector = word_embeddings + positional_embeddings

        x = self.blocks(final_vector) # (B,T,C)
        x = self.ln1(x) # (B,T,C)
        logits = self.lm_head(x) # (B,T,vocab_size)
        num_samples, num_words, num_embeddings = logits.shape
        
        if targets == None:
            loss = None
        else:
            logits = logits.view(num_samples * num_words, num_embeddings)
            targets = targets.view(num_samples * num_words)
            loss = nn.functional.cross_entropy(logits, targets)
        return logits, loss

    def generate(self, idx, max_new_tokens):
        # idx is (B, T) array of indices in the current context
        for _ in range(max_new_tokens):
            # crop idx to the last block_size tokens
            idx_cond = idx[:, -self.block_size:]
            # get the predictions
            logits, loss = self(idx_cond)
            # focus only on the last time step
            logits = logits[:, -1, :] # becomes (B, C)
            # apply softmax to get probabilities
            probs = nn.functional.softmax(logits, dim=-1) # (B, C)
            # sample from the distribution
            idx_next = torch.multinomial(probs, num_samples=1) # (B, 1)
            # append sampled index to the running sequence
            idx = torch.cat((idx, idx_next), dim=1) # (B, T+1)
        return idx

In [11]:
@torch.no_grad()
def estimate_loss(model, train_data, val_data, block_size, n_embd, eval_iters: int = 20):
    # Dictionaries to store the average loss for each split
    out = {}
    model.eval() # Set model to evaluation mode (crucial for dropout/BatchNorm)

    for split_name, data in [('train', train_data), ('val', val_data)]:
        losses = torch.zeros(eval_iters)
        for k in range(eval_iters):
            # Sample data from the current split (train or val)
            xb, yb = sample_data(data, block_size, n_embd)
            # Forward pass to get logits and loss
            logits, loss = model(xb, yb)
            losses[k] = loss.item()
        out[split_name] = losses.mean()

    model.train() # Set model back to training mode
    return out

## 3. Training Optimization: Loss and Gradient Descent

### Loss Function
The model's final layer, the language model head (`lm_head`), produces **logits** of shape $(B, T, V)$. These logits represent the raw, unnormalized scores for every possible next character in the vocabulary at every time step.

The model uses the standard **Cross-Entropy Loss** for classification tasks, comparing the predicted logits against the true target tokens $Y$.

$$\text{Loss} = \text{CrossEntropyLoss}(\text{Logits}, \text{Targets})$$

To calculate this, the logits and targets are often reshaped into two-dimensional vectors, effectively treating all $B \times T$ predictions in the batch as independent classification problems:
* $\text{Logits}: (B \cdot T, V)$
* $\text{Targets}: (B \cdot T)$

### Gradient Descent and Optimization
The goal of training is to find the model parameters (weights) that minimize this loss. This is achieved using **Gradient Descent**, specifically the **AdamW optimizer**.

1.  **Forward Pass:** Calculate the loss.
2.  **Backward Pass (Backpropagation):** The `loss.backward()` call computes the gradient (derivative) of the loss with respect to every single parameter in the model.
3.  **Parameter Update:** The `optimizer.step()` function uses these calculated gradients to update the model's parameters in the direction that decreases the loss, aiming for convergence.

The model, with approximately **0.308329 million parameters**, achieved an initial loss of $4.86$ and dropped to $1.71$ after 1000 steps.

---

In [14]:
num_layers = 6
n_embd = 64
num_heads = 12
block_size = 32
gpt = NanoGPT(num_layers, vocab_size, n_embd, num_heads, block_size)
print(sum(p.numel() for p in gpt.parameters())/1e6, 'M parameters')

0.308329 M parameters


In [99]:
optimizer = torch.optim.AdamW(gpt.parameters(), lr=1e-3)
for steps in range(1000): # increase number of steps for good results...
    # sample a batch of data

    xb, yb = sample_data(training_data, block_size, n_embd)

    # evaluate the loss
    logits, loss = gpt(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()
    
    if steps % 100 == 0:
        print(loss.item())

print(loss.item())

4.8657708168029785
2.3796846866607666
2.2073700428009033
2.044827699661255
1.935045599937439
1.8543068170547485
1.9088845252990723
1.8190929889678955
1.7947200536727905
1.7648370265960693
1.7102956771850586


In [100]:
print(decode(gpt.generate(idx = torch.zeros((1, 1), dtype=torch.long), max_new_tokens=1000)[0].tolist()))


demore a ella vecín, por confería, se lá poder por cemo vol estido hasto le dejado.

— No, se
elló —rinos sevillos fue cosamentia, amo su soyo de rudir se todor quien a llamantillap unto! — que se de vuestro!,
y a mer como Sancho. Fermo que va
esto, puje don Quamó tuficón don Quijote que me soy promera don Quijote de más
mujorsión a jocas que eja y o caballere,
querá la
demas al que se le humbre a mi ínsensa hos y ni fuermar e tumbizna don Jurad a dal en pinsó y lo que habita?

»¡¡A armásfuda de nola
pellerír en nuego los apreso para a con acorgarle.
 Quéjote, os de
así dio a mí podre adellí 
que; ¡Cos Da por desea le que dir aquel con la vanta a masmo, mis
alizo no este criéndono sabijo, antuendo venía o dos que curra, a estaba, aunque mi se
erán estracas, y que mi los suceles pensaco y al cuanto.
El roblar en los pliguesa
de cambieno, que sabía
irtento; dos farre a su está trengándo el corracído
el más de
camina ha del Bistar la que homos puceda, otro de
vuelcarrime, que lo qué tían

## 4. Why Character-Level Tokenization is Inefficient

The character-level approach, while simple and robust against out-of-vocabulary (OOV) words, is highly **inefficient** for training and running modern Transformer-based LLMs:

1.  **Excessive Sequence Length (Sequence Bloat):** A single common word requires multiple tokens (characters). This maximizes the sequence length ($T$) for any given text. Texts encoded at the character level are often **3 to 5 times longer** than those encoded using subword methods.
2.  **Quadratic Computational Cost:** The self-attention mechanism, the core operation of the Transformer, has a computational complexity that is **quadratic** with respect to the sequence length, $O(T^2)$. If the sequence length is four times longer, the computation for attention is $\sim 16$ times slower.
3.  **Reduced Effective Context Window:** Since LLMs have a fixed maximum context length, excessive tokenization limits the number of actual words the model can "see" at once, hindering its ability to capture **long-range dependencies** necessary for high-quality language understanding and generation.
4.  **Low Information Density:** Individual character tokens carry very **low semantic meaning**. The model must dedicate substantial capacity to learning the basic compositional structure of words rather than focusing on high-level linguistic relationships.

---

## 5. Next Steps: Implementing Byte Pair Encoding (BPE)

To overcome the inefficiencies of character-level tokenization, an implementation of the **Byte Pair Encoding (BPE)** algorithm was written as the next step. BPE, as used in models like GPT-2, balances character-level flexibility with word-level efficiency.

The BPE implementation in the notebook followed the general algorithm described on **Wikipedia**:
1.  **Iterative Merge:** Start with the input string represented by bytes.
2.  **Substitution:** Recursively identify the most frequent adjacent pair of tokens and replace them with a new, single token ID (starting from value 256).
3.  **Dictionary:** The substitution is recorded in a `dictionary` mapping the new token ID to the two tokens it replaces, allowing for decoding.

### Note on GPT-2 Paper and BPE
The implemented BPE was a pure algorithmic approach and **did not** incorporate the specific byte-level exception mentioned in the GPT-2 paper. This exception is typically added to prevent the merging of common words with preceding or trailing punctuation (e.g., separating `dog` from `.`, `!`, or `?`) to preserve valuable vocabulary slots and improve compression efficiency.

---

In [6]:
def bp_encoding(input_bytes:list[bytes], dictionary={}, unused=256, level=0, max_level=20)-> list[int]:
    len_bytes = len(input_bytes)
    byte_pair_bin = {}

    # here walk through the input and calculate the byte pairs
    for i in range(len_bytes-1):
        packed_bp = input_bytes[i] | (input_bytes[i+1] << 32)

        if packed_bp not in byte_pair_bin:
            byte_pair_bin[packed_bp] = 1
        else:
            byte_pair_bin[packed_bp] += 1

    # here we walk through the input in order to replace the byte pair
    index = 0
    top_bp = max(byte_pair_bin, key=byte_pair_bin.get)

    if byte_pair_bin[top_bp]== 1 or level >= max_level:
        return input_bytes
    
    encoded = []
    update_unused = False
    while index < len_bytes:
        if index == len_bytes - 1:
            encoded.append(input_bytes[index])
            break
        
        packed_bp = input_bytes[index] | (input_bytes[index+1] << 32)

        if packed_bp == top_bp:
            encoded.append(unused)
            dictionary[unused] = top_bp
            index += 2
            update_unused = True
        else:
            encoded.append(input_bytes[index])
            index += 1
    
    if update_unused:
        unused += 1
    
    return bp_encoding(encoded, dictionary, unused, level + 1, max_level)

def bp_encode(input_str:str, max_level=20) -> [str, dict[int,int]]:
    dictionary={}
    input_bytes = [int(b) for b in input_str.encode("utf-8")]

    return bp_encoding(input_bytes, dictionary, max_level=max_level), dictionary

In [28]:
def bp_decode(input_tokens: list[int], dictionary: dict) -> list[int]:
    decoded_tokens = list(input_tokens)

    # Sort the dictionary keys (new symbols) in descending order.
    # We must reverse the substitutions, so we start with the last encoded symbol.
    symbols_to_expand = sorted(dictionary.keys(), reverse=True)

    for symbol in symbols_to_expand:
        # Get the packed pair integer for the current symbol
        packed_pair = dictionary[symbol]
        
        # Token_2 (MSB): Shift right by 32
        # Token_1 (LSB): Use bitwise AND with 0xFFFF to isolate the lower 32 bits
        token_2 = packed_pair >> 32
        token_1 = packed_pair & 0xFFFFFFFF
        original_pair = [token_1, token_2] # Note: Order is LSB then MSB

        new_decoded_tokens = []
        i = 0
        while i < len(decoded_tokens):
            if decoded_tokens[i] == symbol:
                # Replace the encoded symbol with its original two tokens
                new_decoded_tokens.extend(original_pair)
            else:
                new_decoded_tokens.append(decoded_tokens[i])
            i += 1
        
        decoded_tokens = new_decoded_tokens

    return bytes(decoded_tokens).decode("utf-8")

In [31]:
input_str = "😀😃😄😁😆😅🤣😂🙂🙃"
print(input_str)
e, dictionary = bp_encode(input_str, max_level=25)
rev_dictionary = {v:k for k,v in dictionary.items()}
print(e)
print(dictionary)
print(rev_dictionary)
print(bp_decode(e, dictionary))

😀😃😄😁😆😅🤣😂🙂🙃
[257, 128, 257, 131, 257, 132, 257, 129, 257, 134, 257, 133, 256, 164, 163, 257, 259, 259, 131]
{256: 682899800304, 257: 652835029248, 258: 1099511627906, 259: 657129996546}
{682899800304: 256, 652835029248: 257, 1099511627906: 258, 657129996546: 259}
😀😃😄😁😆😅🤣😂🙂🙃
